<a href="https://colab.research.google.com/github/raspberryscorn/JNU_ddeep/blob/main/3_10_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Model and Dataset
로컬 PC에 저장되어 있는 소스파일을 github에 업로드한 후 colab에서 github에 저장된 소스파일 다운로드


In [ ]:
!wget https://github.com/raspberryscorn/3_10_3/releases/download/cancer/data.zip

# Import
프로그램에 필요한 모듈 선언

In [ ]:
import random
import os
import time
import zipfile

import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras

# Unzip Dataset
다운로드 받은 소스파일 압축 해제

In [ ]:
with zipfile.ZipFile("data.zip", "r") as f:
    f.extractall('./')

# Unet

In [ ]:
# 인공신경망을 제작합니다.
class Unet():
  def graph(input_X, input_Y):
      input = keras.layers.Input((input_X, input_Y, 3))

      # 첫 번째 Convolution Block
      Conv1 = keras.layers.Conv2D(16, (3, 3), activation="relu", padding='same')(input)
      Conv1 = keras.layers.Conv2D(16, (3, 3), activation="relu", padding='same')(Conv1)
      Pool1 = keras.layers.MaxPooling2D((2, 2)) (Conv1)

      # 두 번째 Convolution Block
      Conv2 = keras.layers.Conv2D(32, (3, 3), activation="relu", padding='same')(Pool1)
      Conv2 = keras.layers.Conv2D(32, (3, 3), activation="relu", padding='same')(Conv2)
      Pool2 = keras.layers.MaxPooling2D((2, 2))(Conv2)

      # 세 번째 Convolution Block
      Conv3 = keras.layers.Conv2D(64, (3, 3), activation="relu", padding='same')(Pool2)
      Conv3 = keras.layers.Conv2D(64, (3, 3), activation="relu", padding='same')(Conv3)
      Pool3 = keras.layers.MaxPooling2D((2, 2))(Conv3)

      # 네 번째 Convolution Block
      Conv4 = keras.layers.Conv2D(128, (3, 3), activation="relu", padding='same')(Pool3)
      Conv4 = keras.layers.Conv2D(128, (3, 3), activation="relu", padding='same')(Conv4)
      Pool4 = keras.layers.MaxPooling2D((2, 2))(Conv4)

      # 다섯 번째 Convolution Block
      Conv5 = keras.layers.Conv2D(256, (3, 3), activation="relu", padding='same')(Pool4)
      Conv5 = keras.layers.Conv2D(256, (3, 3), activation="relu", padding='same')(Conv5)

      # 첫 번째 Upsampling Block
      Ups1 = keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding="same")(Conv5)
      Ups1 = keras.layers.Concatenate()([Ups1, Conv4])
      Ups1_conv = keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same")(Ups1)
      Ups1_conv = keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same")(Ups1_conv)

      # 두 번째 Upsampling Block
      Ups2 = keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(Ups1_conv)
      Ups2 = keras.layers.Concatenate()([Ups2, Conv3])
      Ups2_conv = keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(Ups2)
      Ups2_conv = keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(Ups2_conv)

      # 세 번째 Upsampling Block
      Ups3 = keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same")(Ups2_conv)
      Ups3 = keras.layers.Concatenate()([Ups3, Conv2])
      Ups3_conv = keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(Ups3)
      Ups3_conv = keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(Ups3_conv)

      # 네 번째 Upsampling Block
      Ups4 = keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding="same")(Ups3_conv)
      Ups4 = keras.layers.Concatenate()([Ups4, Conv1])
      Ups4_conv = keras.layers.Conv2D(16, (3, 3), activation="relu", padding="same")(Ups4)
      Ups4_conv = keras.layers.Conv2D(16, (3, 3), activation="relu", padding="same")(Ups4_conv)

      output_logit = keras.layers.Conv2D(1, (1, 1))(Ups4_conv)

      return keras.Model(inputs=input, outputs=output_logit)

# Datareader

In [ ]:
# 데이터를 떠먹여 줄 클래스를 제작합니다.
class DataReader():
    def __init__(self):
        self.label = ["Background", "Tumor"]

        self.train_X = []
        self.train_Y = []
        self.test_X = []
        self.test_Y = []

        self.read_data()

    def read_data(self):
        print("Reading Data...")
        images = os.listdir("data/images")
        annotations = os.listdir("data/annotations")

        images.sort()
        annotations.sort()

        data = []

        for i in range(len(images)):
            img = Image.open("data/images/" + images[i])
            ant = Image.open("data/annotations/" + annotations[i])

            if img.mode != "RGB":
                img = img.convert("RGB")

            X = np.asarray(img) / 255.0

            Y_temp = np.asarray(ant)[:, :, 0]
            Y = np.zeros_like(Y_temp)
            Y[Y_temp > 127.5] = 1.0

            data.append((X, Y))
            img.close()
            ant.close()

        random.shuffle(data)

        for i, el in enumerate(data):
            if i < 0.8*len(data):
                self.train_X.append(el[0])
                self.train_Y.append(el[1])
            else:
                self.test_X.append(el[0])
                self.test_Y.append(el[1])

        self.train_X = np.asarray(self.train_X)
        self.train_Y = np.asarray(self.train_Y)
        self.test_X = np.asarray(self.test_X)
        self.test_Y = np.asarray(self.test_Y)

        # 데이터 읽기가 완료되었습니다.
        # 읽어온 데이터의 정보를 출력합니다.
        print("\n\nData Read Done!")
        print("Training X Size : " + str(self.train_X.shape))
        print("Training Y Size : " + str(self.train_Y.shape))
        print("Test X Size : " + str(self.test_X.shape))
        print("Test Y Size : " + str(self.test_Y.shape) + '\n\n')

    def show_processed_image(self):
        plt.figure(figsize=(15, 15))
        image = self.train_X[0]
        annotation = self.train_Y[0]
        plt.subplot(1, 2, 1)
        plt.title("images")
        plt.imshow(image)
        plt.subplot(1, 2, 2)
        plt.title("annotation")
        plt.imshow(annotation)


def save_segmentation_results(image, mask_y, graph):
    prediction = graph.predict(image)[:, :, :, 0]
    prediction[prediction < 0] = 0
    pred_mask = (np.copy(image)*255).astype(np.uint8)[:, :, :, 0]
    pred_mask[prediction > 0.5] = 255

    mask = (np.copy(image)*255).astype(np.uint8)[:, :, :, 0]
    mask[mask_y > 0.5] = 255

    image = (image * 255).astype(np.uint8)

    template = np.copy(image)[:, :, :, 0]

    mask = np.stack((template, mask, template), axis=3)
    pred_mask = np.stack((template, template, pred_mask), axis=3)

    if "results" not in os.listdir():
        os.mkdir("results")

    for i in range(len(image)):
        new_canvas = np.concatenate((image[i], mask[i], pred_mask[i]), axis=1)
        img = Image.fromarray(new_canvas)
        img.save("results/" + str(time.time()) + ".jpg")
        img.close()

    print("RESULT SAVED")


def draw_graph(history):
    train_history = history.history["loss"]
    validation_history = history.history["val_loss"]
    fig = plt.figure(figsize=(8, 8))
    plt.title("Loss History")
    plt.xlabel("EPOCH")
    plt.ylabel("LOSS Function")
    plt.plot(train_history, "red")
    plt.plot(validation_history, 'blue')
    fig.savefig("train_history.png")

    train_history = history.history["accuracy"]
    validation_history = history.history["val_accuracy"]
    fig = plt.figure(figsize=(8, 8))
    plt.title("Accuracy History")
    plt.xlabel("EPOCH")
    plt.ylabel("Accuracy")
    plt.plot(train_history, "red")
    plt.plot(validation_history, 'blue')
    fig.savefig("accuracy_history.png")

# Main

In [ ]:
# 몇 에포크 만큼 학습을 시킬 것인지 결정합니다.
EPOCHS = 50  # 예제 기본값은 50입니다.

# 데이터를 읽어옵니다.
dr = DataReader()

# U-Net을 불러옵니다.
model = Unet.graph(128, 128)

# 인공신경망을 컴파일합니다.
loss = keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss, metrics=['accuracy'])

# 인공신경망을 학습시킵니다.
print("\n\n************ TRAINING START ************ ")
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(dr.train_X, dr.train_Y, epochs=EPOCHS,
                    validation_data=(dr.test_X, dr.test_Y),
                    callbacks=[early_stop])

# Segmentation 결과를 저장합니다.
save_segmentation_results(dr.test_X, dr.test_Y, model)

# 학습 결과를 그래프로 출력합니다.
draw_graph(history)